# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,1.08,83,99,6.92,GS,1701111159
1,1,punta arenas,-53.1500,-70.9167,15.01,47,0,5.14,CL,1701111159
2,2,hasaki,35.7333,140.8333,15.22,79,37,9.91,JP,1701111160
3,3,chaoyang,41.5703,120.4586,-7.42,42,0,2.25,CN,1701111160
4,4,calvinia,-31.4707,19.7760,24.76,17,4,2.55,ZA,1701111160


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
hum_map = city_data_df.hvplot.points(
"Lng", "Lat", 
geo = True, 
size = "Humidity", 
scale = 1.5, 
color = "City", 
alpha = 0.5, 
tiles = "OSM", 
frame_width = 800, 
frame_height = 600).opts(xlabel = "Longitude", ylabel = "Latitude")
# Display the map
hum_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filters = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) & 
                           (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)] 

# Drop any rows with null values
dropped = filters.dropna()

# Display sample data
dropped

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
196,196,tidjikja,18.5564,-11.4271,24.64,13,0,3.62,MR,1701111184
207,207,taoudenni,22.6783,-3.9836,21.45,14,0,3.94,ML,1701111186
223,223,kidal,18.4411,1.4078,25.59,13,0,4.39,ML,1701111189
248,248,toliara,-23.3500,43.6667,24.12,89,0,3.87,MG,1701111192
323,323,saint-pierre,-21.3393,55.4781,23.82,78,0,2.57,RE,1701111140
420,420,al qabil,22.5710,58.6947,22.12,60,0,2.84,OM,1701111214
440,440,mandvi,22.8333,69.3667,21.71,70,0,4.48,IN,1701111216
519,519,al buraymi,24.2509,55.7931,22.68,64,0,3.09,OM,1701111225
545,545,al hadd,26.2456,50.6542,25.99,78,0,4.12,BH,1701111228


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = dropped[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
196,tidjikja,MR,18.5564,-11.4271,13,
207,taoudenni,ML,22.6783,-3.9836,14,
223,kidal,ML,18.4411,1.4078,13,
248,toliara,MG,-23.3500,43.6667,89,
323,saint-pierre,RE,-21.3393,55.4781,78,
420,al qabil,OM,22.5710,58.6947,60,
440,mandvi,IN,22.8333,69.3667,70,
519,al buraymi,OM,24.2509,55.7931,64,
545,al hadd,BH,26.2456,50.6542,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
          "apiKey": geoapify_key,
          "limit":20
                    }
# Print a message to follow up the hotel search
print("Starting hotel search")
# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
# Display sample data
hotel_df

Starting hotel search
tidjikja - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
toliara - nearest hotel: Ambary


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
hot_map = hotel_df.hvplot.points(
"Lng", "Lat", 
geo = True, 
size = "Humidity", 
scale = 1.5, 
color = "City", 
alpha = 0.5, 
tiles = "OSM", 
frame_width = 800, 
frame_height = 600,
hover_cols = ["Country", "Hotel Name"]).opts(xlabel = "Longitude", ylabel = "Latitude")
# Display the map
hot_map
